## Polyhedral Compiler Hands-on

Matmul optimization on cpu

In [1]:
import sys
sys.path.append("../")

import caten.isl as I
import caten.polyhedral as P

ImportError: cannot import name 'a' from 'caten.polyhedral.analysis' (/Users/hikettei/Programming/Caten.py/examples/../caten/polyhedral/analysis.py)

## 2. Gemm Optimization

Repro of this article
https://salykova.github.io/matmul-cpu

In [2]:
def create_gemm_schedule():
    A, B, C = map(I.expr, ("A", "B", "C"))
    zero = I.expr(0)

    with P.parameter("M, N, K"):
        with P.domain("{ WMMA[i,j,k] : 0<=i<M and 0<=j<N and 0<=k<K }") as gemm:
            with P.band("{ WMMA[i,j,k] -> [i, j, k] }"):
                P.stmt("C[j,i] = C[j,i], A[k,i], B[j,k]")[
                    lambda i, j, k: C[j, i].assign(C[j, i] + A[k, i] * B[j, k])
                ]
    return gemm.finalize()

print(create_gemm_schedule())
create_gemm_schedule().viz()

NameError: name 'P' is not defined

In [ ]:
with create_gemm_schedule().editor() as gemm:
    gemm = gemm.domain()
    with gemm.band()["ijk -> ikj"] as gemm: # interchange
        gemm = gemm @ P.Parallel(128, 128, 128)
        gemm = gemm << P.Prefetch("A", 1, 8)
        gemm = gemm << P.prefetch("B", 8, 1)
        gemm = gemm @ P.TensorCore(8, 8, 8, reminder="padding")
        # "Domainに戻ってループを書き換える操作"を作成したい
        

In [ ]:
# TODO
# - [ ] Implement dispatch model
# - [ ] Strip-Mine
# - [ ] Parametric Tile
# - [ ] 